## Classification
Put simply, classification is the task of predicting a label for a given observation. For example: you are given certain physical descriptions of an animal, and your taks is to classify them as either a dog or a cat. Here, we will classify iris flowers.

As we will see later, we will use different classifiers and at the end of this notebook, we will compare them. We will define our accuracy function right now to get it out of the way. We will use a simple accuracy function that returns the ratio of the number of correctly classified observations to the total number of predictions.

In [4]:
findaccuracy(predictedvals,groundtruthvals) = sum(predictedvals.==groundtruthvals)/length(groundtruthvals)

findaccuracy (generic function with 1 method)

In [5]:
using GLMNet
using RDatasets
using MLBase
using Plots
using DecisionTree
using Distances
using NearestNeighbors
using Random
using LinearAlgebra
using DataStructures
using LIBSVM

Get the data first

In [6]:
iris = dataset("datasets", "iris")

Row,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
,Float64,Float64,Float64,Float64,Cat…
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa
6,5.4,3.9,1.7,0.4,setosa
7,4.6,3.4,1.4,0.3,setosa
8,5.0,3.4,1.5,0.2,setosa
9,4.4,2.9,1.4,0.2,setosa


In [7]:
X = Matrix(iris[:,1:4])
irislabels = iris[:,5]

150-element CategoricalArrays.CategoricalArray{String,1,UInt8}:
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 ⋮
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"

In [8]:
X

150×4 Matrix{Float64}:
 5.1  3.5  1.4  0.2
 4.9  3.0  1.4  0.2
 4.7  3.2  1.3  0.2
 4.6  3.1  1.5  0.2
 5.0  3.6  1.4  0.2
 5.4  3.9  1.7  0.4
 4.6  3.4  1.4  0.3
 5.0  3.4  1.5  0.2
 4.4  2.9  1.4  0.2
 4.9  3.1  1.5  0.1
 ⋮              
 6.9  3.1  5.1  2.3
 5.8  2.7  5.1  1.9
 6.8  3.2  5.9  2.3
 6.7  3.3  5.7  2.5
 6.7  3.0  5.2  2.3
 6.3  2.5  5.0  1.9
 6.5  3.0  5.2  2.0
 6.2  3.4  5.4  2.3
 5.9  3.0  5.1  1.8

In [9]:
irislabelsmap = labelmap(irislabels)
y = labelencode(irislabelsmap, irislabels)

150-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 3
 3
 3
 3
 3
 3
 3
 3
 3

In classification, we often want to use some of the data to fit a model, and the rest of the data to validate (commonly known as `training` and `testing` data). We will get this data ready now so that we can easily use it in the rest of this notebook.

In [10]:
function perclass_splits(y,at)
    uids = unique(y)
    keepids = []
    for ui in uids
        curids = findall(y.==ui)
        rowids = randsubseq(curids, at) 
        push!(keepids,rowids...)
    end
    return keepids
end

perclass_splits (generic function with 1 method)

In [11]:
?randsubseq

Base.Meta.ParseError: ParseError:
# Error @ c:\Users\chadi\OneDrive - Handelshögskolan i Stockholm\GitHub\JuliaAcademy\DataScience\06. Classification.ipynb:1:1
?randsubseq
╙ ── not a unary operator

In [12]:
trainids = perclass_splits(y,0.7)
testids = setdiff(1:length(y),trainids)

42-element Vector{Any}:
   4
   7
   8
   9
  15
  18
  23
  24
  32
  36
   ⋮
 110
 111
 118
 120
 122
 125
 128
 131
 144

We will need one more function, and that is the function that will assign classes based on the predicted values when the predicted values are continuous.

In [13]:
assign_class(predictedvalue) = argmin(abs.(predictedvalue .- [1,2,3]))

assign_class (generic function with 1 method)

### 🟣 Method 1: Lasso

In [14]:
path = glmnet(X[trainids,:], y[trainids])
cv = glmnetcv(X[trainids,:], y[trainids])

Least Squares GLMNet Cross Validation
72 models for 4 predictors in 10 folds
Best λ 0.001 (mean loss 0.051, std 0.004)

In [15]:
# choose the best lambda to predict with.
path = glmnet(X[trainids,:], y[trainids])
cv = glmnetcv(X[trainids,:], y[trainids])
mylambda = path.lambda[argmin(cv.meanloss)]

path = glmnet(X[trainids,:], y[trainids],lambda=[mylambda]);

In [16]:
q = X[testids,:];
predictions_lasso = GLMNet.predict(path,q)

42×1 Matrix{Float64}:
 1.0040992769561932
 1.0361161770051204
 0.9523291283672816
 1.0050551619598065
 0.7730252863195113
 0.9731927824309166
 0.8745734455148836
 1.1687043679649192
 1.0209576717740387
 0.8778668861605605
 ⋮
 3.200872930452052
 2.7419953817495664
 3.111155867284884
 2.489403278349563
 2.8061798949993846
 2.927186091861226
 2.624159111092738
 2.8258288298054293
 3.0843379624584486

In [17]:
predictions_lasso = assign_class.(predictions_lasso)
findaccuracy(predictions_lasso,y[testids])

0.9523809523809523

### 🟣 Method 2: Ridge
We will use the same function but set alpha to zero.

In [18]:
# choose the best lambda to predict with.
path = glmnet(X[trainids,:], y[trainids],alpha=0);
cv = glmnetcv(X[trainids,:], y[trainids],alpha=0)
mylambda = path.lambda[argmin(cv.meanloss)]
path = glmnet(X[trainids,:], y[trainids],alpha=0,lambda=[mylambda]);
q = X[testids,:];
predictions_ridge = GLMNet.predict(path,q)
predictions_ridge = assign_class.(predictions_ridge)
findaccuracy(predictions_ridge,y[testids])

0.9761904761904762

### 🟣 Method 3: Elastic Net
We will use the same function but set alpha to 0.5 (it's the combination of lasso and ridge).

In [19]:
# choose the best lambda to predict with.
path = glmnet(X[trainids,:], y[trainids],alpha=0.5);
cv = glmnetcv(X[trainids,:], y[trainids],alpha=0.5)
mylambda = path.lambda[argmin(cv.meanloss)]
path = glmnet(X[trainids,:], y[trainids],alpha=0.5,lambda=[mylambda]);
q = X[testids,:];
predictions_EN = GLMNet.predict(path,q)
predictions_EN = assign_class.(predictions_EN)
findaccuracy(predictions_EN,y[testids])

0.9523809523809523

### 🟣 Method 4: Decision Trees
We will use the package `DecisionTree`

In [20]:
model = DecisionTreeClassifier(max_depth=2)
DecisionTree.fit!(model, X[trainids,:], y[trainids])

DecisionTreeClassifier
max_depth:                2
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  [1, 2, 3]
root:                     Decision Tree
Leaves: 3
Depth:  2

In [21]:
q = X[testids,:];
predictions_DT = DecisionTree.predict(model, q)
findaccuracy(predictions_DT,y[testids])

0.9523809523809523

### 🟣 Method 5: Random Forests
The `RandomForestClassifier` is available through the `DecisionTree` package as well.

In [22]:
model = RandomForestClassifier(n_trees=20)
DecisionTree.fit!(model, X[trainids,:], y[trainids])

RandomForestClassifier
n_trees:             20
n_subfeatures:       -1
partial_sampling:    0.7
max_depth:           -1
min_samples_leaf:    1
min_samples_split:   2
min_purity_increase: 0.0
classes:             [1, 2, 3]
ensemble:            Ensemble of Decision Trees
Trees:      20
Avg Leaves: 5.9
Avg Depth:  4.3

In [23]:
q = X[testids,:];
predictions_RF = DecisionTree.predict(model, q)
findaccuracy(predictions_RF,y[testids])

0.9761904761904762

### 🟣 Method 6: Using a Nearest Neighbor method
We will use the `NearestNeighbors` package here.

In [24]:
Xtrain = X[trainids,:]
ytrain = y[trainids]
kdtree = KDTree(Xtrain')

KDTree{StaticArraysCore.SVector{4, Float64}, Euclidean, Float64, StaticArraysCore.SVector{4, Float64}}
  Number of points: 108
  Dimensions: 4
  Metric: Euclidean(0.0)
  Reordered: true

In [25]:
queries = X[testids,:]

42×4 Matrix{Float64}:
 4.6  3.1  1.5  0.2
 4.6  3.4  1.4  0.3
 5.0  3.4  1.5  0.2
 4.4  2.9  1.4  0.2
 5.8  4.0  1.2  0.2
 5.1  3.5  1.4  0.3
 4.6  3.6  1.0  0.2
 5.1  3.3  1.7  0.5
 5.4  3.4  1.5  0.4
 5.0  3.2  1.2  0.2
 ⋮              
 7.2  3.6  6.1  2.5
 6.5  3.2  5.1  2.0
 7.7  3.8  6.7  2.2
 6.0  2.2  5.0  1.5
 5.6  2.8  4.9  2.0
 6.7  3.3  5.7  2.1
 6.1  3.0  4.9  1.8
 7.4  2.8  6.1  1.9
 6.8  3.2  5.9  2.3

In [26]:
idxs, dists = knn(kdtree, queries', 5, true)

([[35, 22, 23, 3, 34], [35, 3, 8, 31, 22], [1, 19, 4, 8, 21], [28, 31, 10, 35, 9], [25, 12, 11, 13, 7], [1, 29, 4, 21, 20], [3, 27, 29, 31, 35], [19, 32, 20, 15, 18], [15, 20, 21, 7, 36], [2, 3, 29, 26, 21]  …  [85, 75, 95, 83, 87], [84, 103, 71, 95, 87], [106, 81, 104, 97, 82], [91, 95, 85, 87, 75], [54, 61, 50, 105, 79], [70, 102, 79, 108, 80], [84, 78, 100, 73, 103], [98, 88, 108, 52, 86], [75, 71, 87, 90, 95], [84, 103, 100, 73, 71]], [[0.14142135623730964, 0.17320508075688812, 0.22360679774997916, 0.24494897427831802, 0.26457513110645925], [0.22360679774997871, 0.264575131106459, 0.3000000000000002, 0.3162277660168373, 0.31622776601683805], [0.17320508075688762, 0.22360679774997902, 0.22360679774997916, 0.22360679774997916, 0.22360679774997916], [0.14142135623730948, 0.31622776601683816, 0.3464101615137755, 0.3605551275463988, 0.42426406871192807], [0.412310562561766, 0.46904157598234253, 0.5477225575051664, 0.5567764362830022, 0.5830951894845297], [0.09999999999999998, 0.14142135

In [27]:
c = ytrain[hcat(idxs...)]
possible_labels = map(i->counter(c[:,i]),1:size(c,2))
predictions_NN = map(i->parse(Int,string(string(argmax(possible_labels[i])))),1:size(c,2))
findaccuracy(predictions_NN,y[testids])

0.9523809523809523

### 🟣 Method 7: Support Vector Machines
We will use the `LIBSVM` package here.

In [28]:
Xtrain = X[trainids,:]
ytrain = y[trainids]

108-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 3
 3
 3
 3
 3
 3
 3
 3
 3

In [29]:
model = svmtrain(Xtrain', ytrain)

LIBSVM.SVM{Int64, LIBSVM.Kernel.KERNEL}(SVC, LIBSVM.Kernel.RadialBasis, nothing, 4, 108, 3, [1, 2, 3], Int32[1, 2, 3], Float64[], Int32[], LIBSVM.SupportVectors{Vector{Int64}, Matrix{Float64}}(36, Int32[5, 15, 16], [1, 1, 1, 1, 1, 2, 2, 2, 2, 2  …  3, 3, 3, 3, 3, 3, 3, 3, 3, 3], [4.3 5.7 … 6.5 5.9; 3.0 4.4 … 3.0 3.0; 1.1 1.5 … 5.2 5.1; 0.1 0.4 … 2.0 1.8], Int32[10, 11, 17, 30, 33, 37, 39, 41, 44, 46  …  91, 93, 94, 96, 98, 101, 102, 105, 106, 108], LIBSVM.SVMNode[LIBSVM.SVMNode(1, 4.3), LIBSVM.SVMNode(1, 5.7), LIBSVM.SVMNode(1, 4.8), LIBSVM.SVMNode(1, 4.5), LIBSVM.SVMNode(1, 5.1), LIBSVM.SVMNode(1, 7.0), LIBSVM.SVMNode(1, 6.9), LIBSVM.SVMNode(1, 4.9), LIBSVM.SVMNode(1, 5.0), LIBSVM.SVMNode(1, 6.1)  …  LIBSVM.SVMNode(1, 7.9), LIBSVM.SVMNode(1, 6.3), LIBSVM.SVMNode(1, 6.1), LIBSVM.SVMNode(1, 6.3), LIBSVM.SVMNode(1, 6.0), LIBSVM.SVMNode(1, 6.9), LIBSVM.SVMNode(1, 5.8), LIBSVM.SVMNode(1, 6.3), LIBSVM.SVMNode(1, 6.5), LIBSVM.SVMNode(1, 5.9)]), 0.0, [0.0 0.008937516838437798; 0.6305577655011

In [30]:
predictions_SVM, decision_values = svmpredict(model, X[testids,:]')
findaccuracy(predictions_SVM,y[testids])

0.9761904761904762

Putting all the results together:

In [31]:
overall_accuracies = zeros(7)
methods = ["lasso","ridge","EN", "DT", "RF","kNN", "SVM"]
ytest = y[testids]
overall_accuracies[1] = findaccuracy(predictions_lasso,ytest)
overall_accuracies[2] = findaccuracy(predictions_ridge,ytest)
overall_accuracies[3] = findaccuracy(predictions_EN,ytest)
overall_accuracies[4] = findaccuracy(predictions_DT,ytest)
overall_accuracies[5] = findaccuracy(predictions_RF,ytest)
overall_accuracies[6] = findaccuracy(predictions_NN,ytest)
overall_accuracies[7] = findaccuracy(predictions_SVM,ytest)
hcat(methods, overall_accuracies)

7×2 Matrix{Any}:
 "lasso"  0.952381
 "ridge"  0.97619
 "EN"     0.952381
 "DT"     0.952381
 "RF"     0.97619
 "kNN"    0.952381
 "SVM"    0.97619

# Finally...
After finishing this notebook, you should be able to:
- [ ] split your data into training and testing data to test the effectiveness of a certain method
- [ ] apply a simple accuracy function to test the effectiveness of a certain method
- [ ] run multiple classification algorithms:
    - [ ] LASSO
    - [ ] Ridge
    - [ ] ElasticNet
    - [ ] Decision Tree
    - [ ] Random Forest
    - [ ] Nearest Neighbors
    - [ ] Support Vector Machines

# 🥳 One cool finding

We used multiple methods to run classification on the `iris` dataset which is a dataset of flowers and there are three types of iris flowers in it. We split the data into training and testing and ran our methods. Here is the scoreboard:

| method | accuracy score |
|---|---|
| lasso  |1.0|
| ridge  |1.0|
| EN     |1.0|
| DT     |0.960784|
| RF     |0.980392|
| kNN    |1.0|
| SVM    |1.0|